In [1]:
import requests 
import pandas as pd 
import numpy as np 
import os 
import sys
import datetime 
import math 
from lxml import etree
import time
from selenium import webdriver
import urllib.parse
# GLOBAL VARIABLES 
x_sample_size=1000 #how many movies you want 

In [2]:
#james' code here 
import bs4
import requests
import pandas as pd
# Pages to scrape
url_list = ['https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/'
            ,'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200'
            ,'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400'
            ,'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=600'
            ,'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=800']

# Column names
cols = ['Rank'
        , 'Title'
        ,'Worldwide Lifetime Gross'
        , 'Domestic Lifetime Gross'
        , 'Domestic %'
        , 'Foreign Lifetime Gross'
        , 'Foreign %'
        , 'Year']
data = []

# Scrape table data from URLs
for url in url_list:
    html_content = requests.get(url).text
    soup = bs4.BeautifulSoup(html_content, "lxml")
    
    table = soup.find('table') 
    table_rows = table.findAll('tr')
    
    data.append([[td.findChildren(text=True) for td in tr.findAll("td")] for tr in table_rows])

# data[-1]
# Transform sraped data into a dataframe
df_mojo = pd.DataFrame()

for j in range(0,len(url_list)):
    for i in data[j]:
        df_mojo = df_mojo.append(pd.DataFrame(i).transpose())

df_mojo.columns = cols
df_mojo.drop(columns = 'Rank', inplace=True)
df_mojo.reset_index(drop = True, inplace = True)

df_mojo.head()

,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,Avatar,"$2,847,246,203","$760,507,625",26.7%,"$2,086,738,578",73.3%,2009
1,Avengers: Endgame,"$2,797,501,328","$858,373,000",30.7%,"$1,939,128,328",69.3%,2019
2,Titanic,"$2,201,647,264","$659,363,944",30%,"$1,542,283,320",70%,1997
3,Star Wars: Episode VII - The Force Awakens,"$2,069,521,700","$936,662,225",45.3%,"$1,132,859,475",54.7%,2015
4,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


In [3]:
#for each in mojo, search on rt, click first result, scrape first 5 pages of results, append to df

l_rev_name=[]
l_rev_date=[]
l_rev=[]
l_mojo_movie_name=[]
l_rt_movie_name=[]
driver = webdriver.Chrome(executable_path=os.path.join(os.getcwd(),'chromedriver.exe') ) 
_time=time.time() #used to give updates
for index, row in df_mojo.iterrows(): #dont judge me 
    rt_movie_name=''
    try:
        if time.time() - _time > 600: #every 10 mins
            print('update: current index:'
                , str(index) + '/' + str(df_mojo.shape[0])
                , ' - current title:', str(row['Title'])
                , ' - current time: ' + (datetime.datetime.now()).strftime('%Y-%m-%d %H:%M:%S') 
                )
            #outputs copies in case it goes wrong 
            df_reviews = pd.DataFrame(
                np.column_stack([
                        l_rev_name
                        ,l_rev_date
                        ,l_rev
                        ,l_mojo_movie_name
                        ,l_rt_movie_name         
                        ]), 
                columns=['reviewer_name','review_date','review','mojo_name','rt_name'])
            df_reviews.to_csv(os.path.join(os.getcwd(),'scrape_reviews_'+str(index)+'.csv'))
            _time=time.time()
        driver.get('https://www.rottentomatoes.com/search?search='+urllib.parse.quote(row['Title']))
        #filter to movies, did the hard way since buttons on page are dynamic
        button_list=driver.find_element_by_class_name('search__nav').find_elements_by_class_name('js-search-filter')
        for button in button_list: 
            for attr in button.get_property('attributes'):
                if 'movie' in attr['value']: movie_button=button
        movie_button.click()
        #get first result url
        cats = driver.find_elements_by_xpath('//*[@id="main-page-content"]/div/section[1]/search-page-result-container/search-page-result')
        for cat in cats: 
            for attr in cat.get_property('attributes'):
                if 'movie' in attr['value']: 
                    movie_area=cat.find_element_by_tag_name("ul")
        for aa in movie_area.find_element_by_tag_name("search-page-media-row").find_elements_by_tag_name("a"):
            for attr in aa.get_property('attributes'):
                if 'title' in attr['value'] and 'slot' in attr['name']: ListlinkerHref=aa
        ListlinkerHref=ListlinkerHref.get_attribute("href")
        driver.get(ListlinkerHref  + '/reviews?type=user') #go to new page 
        time.sleep(2) #page load
        # movie name being scraped
        rt_movie_name=driver.title.replace(' - Rotten Tomatoes','') #driver.find_element_by_xpath('//*[@id="main_container"]/div[1]/section/div/div[1]/h2/a').text

        for y in range(4):  #get first 5 pages
            try:
                reviews=driver.find_element_by_id("movieUserReviewsContent")
                review_table=reviews.find_elements_by_tag_name("ul")
                #parse reviews
                for rev in reviews.find_elements_by_tag_name("li"):
                    #now we write to list 
                    try: l_rev_name.append(rev.text.splitlines()[0])
                    except: l_rev_name.append('error')
                    try: l_rev_date.append(rev.text.splitlines()[1])
                    except: l_rev_date.append('error')
                    try: l_rev.append(rev.text.splitlines()[2])
                    except: l_rev.append('error')
                    l_mojo_movie_name.append(row['Title']) #join to watchmojo moviename
                    l_rt_movie_name.append(rt_movie_name)
                #next page
                next_button=driver.find_element_by_xpath('/html/body/div[5]/div[3]/div[2]/section/div/div/nav[3]/button[2]')
                next_button.click()
                time.sleep(2)
            except: 
                l_rev_name.append('error')
                l_rev_date.append('error')
                l_rev.append('no reviews found')
                l_mojo_movie_name.append(row['Title']) #join to watchmojo moviename
                l_rt_movie_name.append(rt_movie_name)
    except: 
        l_rev_name.append('error')
        l_rev_date.append('error')
        l_rev.append('misc error')
        l_mojo_movie_name.append(row['Title']) #join to watchmojo moviename
        l_rt_movie_name.append('error')

update: current index: 18/1000  - current title: The Fate of the Furious  - current time: 2021-11-28 17:38:09
update: current index: 39/1000  - current title: Despicable Me 3  - current time: 2021-11-28 17:48:26
update: current index: 61/1000  - current title: The Lord of the Rings: The Fellowship of the Ring  - current time: 2021-11-28 17:58:44
update: current index: 82/1000  - current title: Spider-Man  - current time: 2021-11-28 18:08:46
update: current index: 104/1000  - current title: Maleficent  - current time: 2021-11-28 18:19:07
update: current index: 124/1000  - current title: The Amazing Spider-Man 2  - current time: 2021-11-28 18:29:08
update: current index: 146/1000  - current title: Moana  - current time: 2021-11-28 18:39:36
update: current index: 167/1000  - current title: Tangled  - current time: 2021-11-28 18:50:03
update: current index: 190/1000  - current title: Detective Chinatown 2  - current time: 2021-11-28 19:00:08
update: current index: 209/1000  - current title

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=96.0.4664.45)


In [ ]:
#write to df
df_reviews = pd.DataFrame(
        np.column_stack([
                l_rev_name
                ,l_rev_date
                ,l_rev
                ,l_mojo_movie_name
                ,l_rt_movie_name         
                ]), 
        columns=['reviewer_name','review_date','review','mojo_name','rt_name'])


In [ ]:
#get parent sitemap 
url = 'https://www.rottentomatoes.com/sitemaps/sitemap.xml'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

r = requests.get(url, headers=headers, timeout=10)
if len(r.text) > 50:
    root = etree.fromstring(r.content)
    body=root.xpath('//ns:sitemap',namespaces={'ns':"http://www.sitemaps.org/schemas/sitemap/0.9"})

In [ ]:
l_movies_xml=[]
l_m_url=[]
for element in body:
    if 'sitemaps/movie_' in element[0].text: #get movies from sitemap
        l_movies_xml.append(element[0].text) #keep list to be safe
        m_url=requests.get(element[0].text, headers=headers, timeout=10)
        m_url_root = etree.fromstring(m_url.content)
        m_url_body=m_url_root.xpath('//ns:url',namespaces={'ns':"http://www.sitemaps.org/schemas/sitemap/0.9"})
        for url in m_url_body: 
            #only get urls, not /pictures, /trailers etc
            if url[0].text.count('/')==4: l_m_url.append(url[0].text)
        print('length of list:', str(len(l_m_url)))
#write url list to df and get sample dataset
df_m_url = pd.DataFrame (l_m_url, columns = ['movie_urls'])
df_m_url_sample=df_m_url.sample(n = x_sample_size).reset_index(drop=True)

In [ ]:
for index, row in df_m_url_sample.iterrows(): #dont judge me 
    print(row['movie_urls'])
    r_m_page=requests.get(row['movie_urls'] + '/reviews', headers=headers, timeout=10)
    # print(r_m_page.text)
    sys.exit()

In [ ]:
for index, row in df_mojo.iterrows(): #dont judge me 
    if index > 10: sys.exit() 
    print(row['Title'], urllib.parse.quote(urllib.parse.quote(row['Title'])))

In [313]:
# ListlinkerHref = driver.find_element_by_xpath('//*[@id="main-page-content"]/div/section[1]/search-page-result-container/search-page-result[1]/ul/search-page-media-row/a[1]').get_attribute("href")
# ListlinkerHref = driver.find_element_by_xpath('/html/body/div[4]/main/div[1]/div/section[1]/search-page-result-container/search-page-result[2]/ul/search-page-media-row')




['slot', 'franchise']
['type', 'franchise']
['data-qa', 'search-result']
['hidden', '']
['slot', 'movie']
['type', 'movie']
['data-qa', 'search-result']
['slot', 'tv']
['type', 'tv']
['data-qa', 'search-result']
['hidden', '']


'https://www.rottentomatoes.com/m/star_wars_episode_vii_the_force_awakens'

In [ ]:
for attr in movie_area.find_elements_by_tag_name("search-page-media-row"):#.get_property('attributes'):
        print([attr['name'], attr['value']])

In [ ]:
ListlinkerHref

In [ ]:
import urllib.parse
#encode string to url
string='Avengers: Endgame'
#now we need selenium
#im using chrome: 96.0.4664.45
from selenium import webdriver
l_rev_name=[]
l_rev_date=[]
l_rev=[]
l_mojo_movie_name=[]
l_rt_movie_name=[]
driver = webdriver.Chrome(executable_path=os.path.join(os.getcwd(),'chromedriver.exe') )  #use chrome driver from git folder
driver.get('https://www.rottentomatoes.com/search?search='+urllib.parse.quote(string))
#filter to movies 
movie_button=driver.find_element_by_xpath('/html/body/div[4]/main/div[1]/div/section[1]/search-page-result-container/nav/ul/li[2]/span')
movie_button.click()
# movie name being scraped
rt_movie_name=driver.find_element_by_xpath('//*[@id="main_container"]/div[1]/section/div/div[1]/h2/a').text
#get first result url
ListlinkerHref = driver.find_element_by_xpath('//*[@id="main-page-content"]/div/section[1]/search-page-result-container/search-page-result[1]/ul/search-page-media-row/a[1]').get_attribute("href")
# driver.find_element_by_xpath('//*[@id="main-page-content"]/div/section[1]/search-page-result-container/search-page-result[1]')
html = driver.get(ListlinkerHref + '/reviews?type=user') #go to reviews page

for y in range(4):  #get first 5 pages
    reviews=driver.find_element_by_id("movieUserReviewsContent")
    review_table=reviews.find_elements_by_tag_name("ul")
    #parse reviews
    for rev in reviews.find_elements_by_tag_name("li"):
        #now we write to list 
        l_rev_name.append(rev.text.splitlines()[0])
        l_rev_date.append(rev.text.splitlines()[1])
        l_rev.append(rev.text.splitlines()[2])
        l_mojo_movie_name.append(string) #join to watchmojo moviename
        l_rt_movie_name.append(rt_movie_name)
    #next page
    next_button=driver.find_element_by_xpath('/html/body/div[5]/div[3]/div[2]/section/div/div/nav[3]/button[2]')
    next_button.click()
    time.sleep(2)

In [ ]:
df_reviews

In [ ]:
df_reviews

In [ ]:
soup=BeautifulSoup(reviews.text,'html.parser')
soup